In [1]:
import pandas as pd
import numpy as np

In [2]:
csv_data = pd.read_csv(r'CleanDataUpdated.csv');

In [3]:
trials = range(1,41)
bpm = [80,100,120]
gestureSize = ["S","M","L"]

In [4]:
csv_data.DistanceCoveredSoFar = csv_data.DistanceCoveredSoFar.astype(float)
csv_data.VelocityMagnitude = csv_data.VelocityMagnitude.astype(float)
csv_data.AngleToBP1 = csv_data.AngleToBP1.astype(float)
csv_data.BPM = csv_data.BPM.astype(int)
csv_data.Position_Y = csv_data.Position_Y.astype(float)

In [5]:
newOutputTable = pd.DataFrame({'Trial':[],'BPM':[],'GestureSize':[], 'MinAngleRegionTwo':[]})

In [18]:
# Here's where we iterate over the arrays we mentioned earlie
trials = [7]
bpm = [100]
gestureSize = ["M"]
for g in gestureSize:
    for b in bpm:
        for i in trials: 
            sliceByTrial = csv_data.loc[csv_data.Trial == i] 
            sliceByGesture = sliceByTrial.loc[sliceByTrial.GestureSize == g] 
            sliceByBPM = sliceByGesture.loc[sliceByGesture.BPM == b] 
            if(sliceByBPM.Trial.size == 0):
               continue
            
            # =======================================
            # ============ REGION 1 =================
            # =======================================
            InitialPoxX = sliceByBPM.iloc[0].Position_X
            InitialPosY = sliceByBPM.iloc[0].Position_Y
            TimeAtInitialPoint = sliceByBPM.iloc[0].TimeRelativeToPrep
            
            globalHighestPointIndex = sliceByBPM.Position_Y.idxmax()
            
            localHighestPointIndex = globalHighestPointIndex - sliceByBPM.index[0] + 1
            
            sliceToGetPositiveYVelocity = sliceByBPM.loc[sliceByBPM.Velocity_Y > 0]
            
            globalIndexFirstPositiveYVelocity = sliceToGetPositiveYVelocity.index[0]
            
            localIndexFirstPositiveYVelocity = globalIndexFirstPositiveYVelocity - sliceByBPM.index[0]
            
            RegionOne = sliceByBPM[localIndexFirstPositiveYVelocity:localHighestPointIndex+1] 
            
            RegionOneX = RegionOne.iloc[0].Position_X
            RegionOneY = RegionOne.iloc[0].Position_Y
            TimeAtRegionOneStart = RegionOne.iloc[0].TimeRelativeToPrep
            
            # =======================================
            # ============ REGION 2 =================
            # =======================================
            
            smallestYPoint_RegionOneIndex = RegionOne.Position_Y.idxmin()
            
            y_positionHighestPoint = sliceByBPM.Position_Y[globalHighestPointIndex]
            
            y_positionLowestPoint  = sliceByBPM.Position_Y[smallestYPoint_RegionOneIndex]
            
            y_distance = np.absolute(y_positionHighestPoint - y_positionLowestPoint)
            
            y_distanceWithinRegion2 = y_distance/2
            
            portionAfterRegionOne = sliceByBPM[localHighestPointIndex+1:]
            
            RegionTwo = portionAfterRegionOne[np.absolute(portionAfterRegionOne.Position_Y - y_positionHighestPoint) < y_distanceWithinRegion2]
            
            RegionTwoX = RegionTwo.iloc[0].Position_X
            RegionTwoY = RegionTwo.iloc[0].Position_Y
            TimeAtRegionTwoStart = RegionTwo.iloc[0].TimeRelativeToPrep
            # =======================================
            # ========== MIN ANGLE R2 ===============
            # =======================================
            
            BasePosX = sliceByBPM.loc[sliceByBPM.ID.str.contains('Interp')].Position_X
            BasePosY = sliceByBPM.loc[sliceByBPM.ID.str.contains('Interp')].Position_Y
            BaseTime = sliceByBPM.loc[sliceByBPM.ID.str.contains('Interp')].TimeRelativeToPrep
                    
            minAngleRegionTwo = min(RegionTwo["AngleToBP1"])
            newOutputTable = newOutputTable.append({'Trial': i,'BPM':b,'GestureSize':g,'MinAngleRegionTwo':minAngleRegionTwo}, ignore_index = True)

In [19]:
InitialPoxX

0.8009729

In [20]:
InitialPosY

-0.35143240000000003

In [21]:
TimeAtInitialPoint

0.0

In [23]:
RegionOneX

0.6031749

In [26]:
InitialPosY - RegionOneY

0.18482890000000002

In [25]:
TimeAtRegionOneStart

0.2

In [27]:
RegionTwoX

0.5829554

In [30]:
RegionOneY - RegionTwoY

-0.39036000000000004

In [29]:
TimeAtRegionTwoStart

0.48

In [31]:
BaseTime

5876    0.75
Name: TimeRelativeToPrep, dtype: float64